In [1]:
from bs4 import BeautifulSoup
import urllib.request
import string
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity 
import numpy as np
import pickle
import itertools
import pandas as pd

In [5]:
# scrap dari digilib

In [11]:
with open('paper_ori.pkl', 'rb') as f:
    paper = pickle.load(f)

In [12]:
len(paper)

328

In [13]:
paper_x = [f[0] for f in paper]
paper_x = list(set(paper_x))
len(paper_x)

328

In [14]:
# preprocessing

In [15]:
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()
stemmer = StemmerFactory().create_stemmer()

In [16]:
with open('processed_paper.pkl', 'rb') as f:
    processed_paper = pickle.load(f)

In [17]:
# generate thesaurus

In [18]:
with open('thesaurus.pkl', 'rb') as f:
    thesaurus = pickle.load(f)

In [19]:
words = [x for x in thesaurus]

In [20]:
# input query

In [80]:
init_query = 'deteksi kendaraan'

In [81]:
# tf_idf

In [82]:
vectorizer = TfidfVectorizer(use_idf=True)
query = init_query
query = query.lower()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
query = query.translate(remove_punctuation_map)
query = stopword.remove(query)
query = query.split()
query = [stemmer.stem(x) for x in query]
query

['deteksi', 'kendara']

In [83]:
max_result = []
x = [' '.join(query)]
paper_tfidf = vectorizer.fit_transform(x + processed_paper)
q = paper_tfidf[0]
result = cosine_similarity(paper_tfidf, q)
idx = np.argsort(-result,axis=0).flatten()    
final = [[num, y[0], x] for num, y in enumerate(result) if y[0] > 0.0]
max_result += final
max_result = sorted(max_result, key=lambda x: x[1], reverse=True)
set_result = set()
new_result = []
for item in max_result:
    if item[0] not in set_result:
        set_result.add(item[0])
        new_result.append(item)
    else:
        pass
len(new_result)

84

In [84]:
for x in new_result[1:5]: 
    print('Result', x[0]) 
    print('QUERY', x[2]) 
    print(paper[x[0]-1][1]) 
    print(paper[x[0]-1][2][:200] + '...')
    print()

Result 213
QUERY ['deteksi kendara']
PROTOTYPE OF EMISSION TEST DEVICE FOR VEHICLE USING ARDUINO MICROCONTROLLER
Belakangan ini Dinas Perhubungan kembali menggalakkan pengujian kendaraan bermotor kepada masyarakat agar masyarakat paham dengan kondisi mesin kendaraannya maupun dampak emisinya terhadap lingkungan ...

Result 214
QUERY ['deteksi kendara']
PROTOTYPE OF EMISSION TEST DEVICE FOR VEHICLE USING ARDUINO MICROCONTROLLER
Belakangan ini Dinas Perhubungan kembali menggalakkan pengujian kendaraan bermotor kepada masyarakat agar masyarakat paham dengan kondisi mesin kendaraannya maupun dampak emisinya terhadap lingkungan ...

Result 76
QUERY ['deteksi kendara']
FORECASTING NUMBER OF VEHICLE IN ROAD USING MULTILAYER PERCEPTRON NEURAL NETWORK WITH AND WITHOUT LINEAR REGRESSION
Jumlah kendaraan yang berada kota-kota besar Indonesia semakin lama semakin bertambah tanpa diimbangi oleh
pertambahan jumlah jalan yang sebanding. Setelah beberapa tahun masalah ini tidak ditangani ...

Result 7

In [85]:
idx_res = [x[0]-1 for x in new_result[1:]]
file = []
for i,x in enumerate(paper):
    if i in idx_res:
        file.append([x[1],x[2],'ok_ok'])
    else:
        file.append([x[1],x[2],''])
df = pd.DataFrame(file)
df.to_excel('hasil/hasil ori ' +init_query+ '.xlsx', header=False, index=False)

In [86]:
# query_expansion

In [87]:
vectorizer = TfidfVectorizer(use_idf=True)
query = init_query
query = query.lower()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
query = query.translate(remove_punctuation_map)
query = stopword.remove(query)
query = query.split()
query = [stemmer.stem(x) for x in query]
query

['deteksi', 'kendara']

In [88]:
product_query = []
list_synonym = []
for x in query:
    if x in words:
        list_synonym.append(thesaurus[x])
    else:
        name = x
        data = { "q": name }
        encoded_data = urllib.parse.urlencode(data).encode("utf-8")
        content = urllib.request.urlopen("http://www.sinonimkata.com/search.php", encoded_data)
        soup = BeautifulSoup(content, 'html.parser')
        try:
            synonym = soup.find('td', attrs={'width': '90%'}).find_all('a')
            synonym = [x.getText() for x in synonym]
            thesaurus[x] = [x] + synonym
            list_synonym.append(thesaurus[x])
        except:
            list_synonym.append([x])
qs = []
for x in itertools.product(*list_synonym):
    x = [stemmer.stem(y) for y in x]
    qs.append([' '.join(x)])
for x in qs:
    print(x, end=' ')

['deteksi kendara'] ['deteksi bawa'] ['deteksi naik'] ['deteksi kemudi'] ['deteksi kendali'] ['deteksi gandar'] ['deteksi jalan'] ['deteksi tumpang'] ['deteksi tunggang'] ['deteksi setir'] ['temu kendara'] ['temu bawa'] ['temu naik'] ['temu kemudi'] ['temu kendali'] ['temu gandar'] ['temu jalan'] ['temu tumpang'] ['temu tunggang'] ['temu setir'] ['indra kendara'] ['indra bawa'] ['indra naik'] ['indra kemudi'] ['indra kendali'] ['indra gandar'] ['indra jalan'] ['indra tumpang'] ['indra tunggang'] ['indra setir'] 

In [89]:
max_result = []
for x in qs:
    paper_tfidf = vectorizer.fit_transform(x + processed_paper)
    q = paper_tfidf[0]
    result = cosine_similarity(paper_tfidf, q)
    idx = np.argsort(-result,axis=0).flatten()    
    final = [[num, y[0], x] for num, y in enumerate(result) if y[0] > 0.0]
    max_result += final
max_result = sorted(max_result, key=lambda x: x[1], reverse=True)
set_result = set()
new_result = []
for item in max_result:
    if item[0] not in set_result:
        set_result.add(item[0])
        new_result.append(item)
    else:
        pass
len(new_result)

186

In [90]:
for x in new_result[1:5]: 
    print('Result', x[0]+1) 
    print('QUERY', x[2]) 
    print(paper[x[0]-1][1]) 
    print(paper[x[0]-1][2][:200] + '...')
    print()

Result 214
QUERY ['temu kendara']
PROTOTYPE OF EMISSION TEST DEVICE FOR VEHICLE USING ARDUINO MICROCONTROLLER
Belakangan ini Dinas Perhubungan kembali menggalakkan pengujian kendaraan bermotor kepada masyarakat agar masyarakat paham dengan kondisi mesin kendaraannya maupun dampak emisinya terhadap lingkungan ...

Result 215
QUERY ['temu kendara']
PROTOTYPE OF EMISSION TEST DEVICE FOR VEHICLE USING ARDUINO MICROCONTROLLER
Belakangan ini Dinas Perhubungan kembali menggalakkan pengujian kendaraan bermotor kepada masyarakat agar masyarakat paham dengan kondisi mesin kendaraannya maupun dampak emisinya terhadap lingkungan ...

Result 311
QUERY ['deteksi kendali']
CONTROLLING ROBOT BASED ON IP INTERNET
PROTOCOL THROUGH THE WIRELESS NETWORK
WITH ANDROID MOBILE DEVICE
Pengembangan teknologi informasi pada saat ini telah
mengalami kemajuan yang sangat pesat khususnya dalam
penelitian mengenai sistem jaringan komputer. Salah satu contoh
hasil pengembangan teknolog...

Result 312
QUERY ['deteksi 

In [91]:
idx_res = [x[0]-1 for x in new_result[1:]]
file = []
for i,x in enumerate(paper):
    if i in idx_res:
        file.append([x[1],x[2],'ok_ok'])
    else:
        file.append([x[1],x[2],''])
df = pd.DataFrame(file)
df.to_excel('hasil/hasil expansion ' +init_query+ '.xlsx', header=False, index=False)